<a href="https://colab.research.google.com/github/quantority/master-s-degree-projects/blob/main/stepik_routine/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [4]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [18]:
# ваш код здесь

data.describe(include='object')

,Education,Occupation,Commute Distance,Region
count,1000,1000,1000,1000
unique,5,5,5,3
top,Bachelors,Professional,0-1 Miles,North America
freq,306,276,366,508


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [16]:
# ваш код здесь

for col in data.select_dtypes(include='object').columns:
  if data[col].nunique() == 2:
    data[col] = data[col].apply(lambda x: 1 if x == data[col].mode().values else 0)
  else:
    pass

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [22]:
# ваш код здесь

data.drop(data.select_dtypes(include='object').columns, axis=1, inplace=True)

Удалите столбец `ID`, так как он по сути нечисловой.

In [24]:
# ваш код здесь
data.drop('ID', axis=1, inplace=True)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [25]:
# ваш код здесь

X = data.drop('Purchased Bike', axis=1).values

y = data['Purchased Bike'].values

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [26]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [27]:
# ваш код здесь

Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [28]:
Xtrain

array([[    1,     1, 50000, ...,     1,     0,    34],
       [    0,     1, 90000, ...,     0,     3,    34],
       [    0,     1, 40000, ...,     0,     0,    36],
       ...,
       [    0,     1, 30000, ...,     1,     1,    32],
       [    0,     0, 10000, ...,     0,     1,    68],
       [    1,     1, 10000, ...,     1,     0,    49]])

In [33]:
from sklearn.preprocessing import MinMaxScaler

# ваш код здесь

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(Xtrain)

X_test_scaled = scaler.transform(Xtest)


Xtrain = pd.DataFrame(X_train_scaled, columns=data.drop('Purchased Bike', axis=1).columns)

Xtest = pd.DataFrame(X_test_scaled, columns=data.drop('Purchased Bike', axis=1).columns)

Теперь обучите логистическую регрессию на тренировочных данных

In [35]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь

lr = LogisticRegression()

lr.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [36]:
# ваш код здесь

predictions = lr.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [37]:
from sklearn.metrics import accuracy_score

# ваш код здесь

accuracy_score(ytest, predictions)

0.5766666666666667

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [38]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь

poly = PolynomialFeatures(degree=2, include_bias=False)

Xtrain = poly.fit_transform(Xtrain)

Xtest = poly.fit_transform(Xtest)

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [41]:
X.shape

(1000, 7)

In [40]:
Xtest.shape

(300, 35)

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [42]:
# ваш код здесь

lr = LogisticRegression()

lr.fit(Xtrain, ytrain)

predictions = lr.predict(Xtest)

accuracy_score(ytest, predictions)

0.62

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [ ]:
# ваш код здесь

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [ ]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [ ]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    # ваш код здесь

print (BestThr, RecMax, BestAcc)

Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [ ]:
# ваш код здесь

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.